In [1]:
import pandas as pd
import numpy as np
import itertools as it 
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, ConstantKernel as C

from ast import literal_eval

# Guide and Ideas for Capstone Project

We begin this guide by downloading the data:

In [ ]:
#PARAMETERS 
# This section defines the variables to run the rest of the document 
file = 1  # Identifies the function number to execute (1,2,3...8) 
resultsFile = "data/results09.csv"   # The latest results file provided by Carlton on a weekly basis 
kernel_name = "polynomial"   # Options are default, linear and polynomial

In [ ]:
inputColumnName = "f" + str(file)
outputColumnName = "f" + str(file) + "_output"
df = pd.read_csv(resultsFile)
input = df[inputColumnName]
output = df[outputColumnName]
print("Input=", input)
print("Output=", output)
df.head(25)


In [ ]:
# LOAD INITIAL DATA PROVIDED AT THE BEGINNING 
def load_data(dim):
    X = np.load('data/initial_data/function_' + str(dim) +'/initial_inputs.npy')
    y = np.load('data/initial_data/function_' + str(dim) +'/initial_outputs.npy')
    return X,y

# LOAD AND APPEND THE RESULTS PUBLISHED BY CARLTON 
X, y = load_data(file)
dimension = np.shape(X)[1] 

for item in input:
     item_with_comma = "[" + item.replace(" 0.",", 0.")  + "]"  
     row = np.array(literal_eval(item_with_comma))
     X = np.append(X, np.array(row), axis=0)

for item in output:
    y = np.append(y, float(item))

print("X(shape)=", X.shape, "y(shape)=", y.shape)


In [ ]:
X

In [ ]:
y


In [ ]:
# The first 2 functions can be shown as X,Y coordinates and reveal what is going on. On higher dimensions it loses meaning so we don't plot it 
if (dimension == 1 or dimension == 2):
    x1 =X[:,0]
    x2 =X[:,1] 
    plt.scatter(x1, x2, c=y)
    for (i, j, text) in zip(x1, x2, y):
        plt.annotate(format(text, ".2e"),  # This is the text to use for the annotation
                 (i, j),  # This is the point to label
                 textcoords="offset points",  # how to position the text
                 xytext=(0,10),  # distance from text to points (x,y)
                 ha='right')  # horizontal alignment can be left, right or center
    plt.show()



# Upper Confidence Bound

A second alternative would be to use Bayesian Optimization and consider an Upper Confidence Bound acquisition function: 

In [ ]:
# Add here hyper-parameters
if (kernel_name == "linear"):
    kernel = C(1.0, constant_value_bounds="fixed") * DotProduct(sigma_0=1.0)
else:
    kernel = C(1.0, constant_value_bounds="fixed") * RBF(1.0, length_scale_bounds="fixed")

gpr = GaussianProcessRegressor(kernel=kernel)
gpr.fit(X, y)



In [ ]:
def find_maxiumum(X, y):
    maximum_of_X = []
    maximum_so_far = -1e99
    for index in range(len(y)):
        if (y[index] > maximum_so_far):
            maximum_so_far = y[index]
            maximum_of_X = []
            for i in range(dimension):
                maximum_of_X.append(X[index, i])
    return np.array(maximum_of_X), maximum_so_far
    
discovery_radius = 0.2   # on each datapoint use 20% discovery radius

Xmax, ymax = find_maxiumum(X, y)  #Returns the row with maximim y and all X values 
XmaxLower = Xmax - (Xmax * discovery_radius)
XmaxUpper = Xmax + (Xmax * discovery_radius)
print("maximum X=", Xmax, ", maximum y=", ymax)

# Adjust sizes as we go up in dimensions to avoid waiting for hours on higher dimensions 
# ********* For the purpose of unblocking any memory issue please decrease these values to adjust to your computer specs ******************
sizes = [ [2, 100],
          [3,  80],
          [4,  60],
          [5,  30],
          [6,  15],
          [7,  10],
          [8,  8]]

size = sizes[dimension - 2][1]  #Find the corrects size for this dimension 

data = []
for item in range(dimension):
    #  Upper lower min and max explanation: 
    #  This iteration (each X value) creates a number of points of size "size" in a radius of +/-20% and ensures this radius is between 0 and 1, e.g.  
    # - X values [0.5, 0, 0.99] 
    # - apply 20% discovery radius => [(0.4 to 0.6), (0 to 0), (0.792 to 1.188)] 
    # - apply 0 to 1 filter => [ linspace(0.4 to 0.6), linspace(0 to 0.0001), linspace(9.792 to 9.9999)]
    # - result = [ [0.48777254 ... 0.59072617], [0.000002, ... 0.000099], [9.7921111, ...9.998888] ]
    if (XmaxUpper[item] == 0):
        XmaxUpper[item] = 0.0001
    if (XmaxLower[item] == 1):
        XmaxLower[item] = 0.9999
    data.append(np.linspace(max(XmaxLower[item], 0 + 1e-10), min(XmaxUpper[item], 1 - 1e-10), size))

x_grid_points = np.array(data)

print("data-points=", x_grid_points.shape)
for dim in range(dimension):
    print("data-points[", dim + 1, "]=", x_grid_points[dim][:5], "...", x_grid_points[dim][-1] )



In [ ]:
# Apply grid search = cartesian product of the array, e.g. data of shape (2,100) => (100 x 100) = (10000, 2)
X_grid = np.fromiter(it.chain(*it.product(*x_grid_points)), dtype=float).reshape(-1,dimension)
print("X_grid=", X_grid.shape, " = ", X_grid[0:5,:], "...", X_grid[-1,:])


In [ ]:
mean, std = gpr.predict(X_grid, return_std = True)

def calculate_ucb(beta):   
    ucb = mean + beta * std
    print("UCB=", ucb.shape, ", value=", ucb[:5])
    print("mean=", mean.shape, ", value=", mean[:5])
    idx_max = np.argmax(ucb)   # take the index of the highest confidence result 
    next_query = X_grid[idx_max]    # next_query is the point with highest confidence in this grid of search 
    best_numbers = []
    for item in next_query:
        best_numbers.append(min(max(item, 0.000001), 0.999999))   # ensure the numbers are between 0 and 1 
    print(best_numbers)
    return best_numbers

def print_results(label, best_numbers):
    #Formats the values in a way it can be copied and pasted into the weekly submission form 
    print(label)
    print(np.array2string(np.array(best_numbers), precision=6, separator='-', floatmode='fixed', formatter={'float': '{:0.6f}'.format}))

print("balanced")
balanced = calculate_ucb(1.96)  #keep it balanced

print("\n\nexploitative")
exploitative = calculate_ucb(1.0)   #lets exploit target area

print("\n\nexploratory")
exploratory = calculate_ucb(2.5)   #lets explore more 



In [ ]:

print_results("exploitative", exploitative)

print_results("\nbalanced", balanced)

print_results("\nexploratory", exploratory)
